In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.utils import shuffle

import sys 
sys.path.append('../src/utils/')
sys.path.append('../src/models/')
sys.path.append('../src/visualization/')

from particles import Photon
from center_finder import CenterFinder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and prepare data for training from one- and two-particle samples

In [5]:
photon_1 = Photon(n_pcl=1)
photon_2 = Photon(n_pcl=2)

photons = [photon_1, photon_2]

In [70]:
seed_paths = {"model_path": "../notebooks/models/seed_finder/run_3/", 
              "weight_path": "../notebooks/models/seed_finder/run_3/weights.27-0.0105.hdf5"}

In [79]:
X, indices, is_seed, y, en, ypr = photon_2.data_for_center_finder("debug", 0.3, **seed_paths)

100%|██████████| 1000/1000 [00:00<00:00, 3670.80it/s]


1040/1040 [==============================] - 4s 3ms/step
